# <font color=blue>  Mapa de Calor  </font>
## <font color=black>  Distribuição de alunos de Engenharia da Computação pelo mapa do DF </font>
#### <font color=black> Escopo de análise delimitado entre o semestre de 2009 até o último semestre de 2018 do curso de Engenharia da Computação da Universidade de Brasília.</font>

In [1]:
#importar bibliotecas

import numpy as np
import pandas as pd
from geopy.geocoders import GoogleV3
import csv
from geopy import geocoders
import json
from pathlib import Path
import pycep_correios
import warnings
import folium
from folium.plugins import HeatMap
import os
import subprocess
import time
import re
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import plotly
import plotly.graph_objs as go
import plotly.plotly as py
import cufflinks

#parametros para rodar API do Google
g = geocoders.GoogleV3(api_key='chave API do Goohgle Maps')
geolocator = GoogleV3()

#ignorar avisos do código
warnings.filterwarnings('ignore')

In [ ]:
path = 'data/anoecep_ira_parte15.csv'
df_cep =  pd.read_csv(path, sep =';')
df_cep = df_cep.fillna(0)
df_cep.head()

#### <font color=black> Transformar dados de CEP para dados de Latitude e Longitude com o uso da API do google maps e API dos correios</font>

In [ ]:
my_file = Path("data/lat_log_uni.json")

if my_file.is_file():
    lat_log = json.load(open('data/lat_log_uni.json'))
else:
    lat_log ={}
    with open("data/cep_uni.txt", "r") as f:
        f = f.read().splitlines() 
        for x in f:

            if not(str(x) in lat_log):
                inputAddress = str(x)
                location = g.geocode(inputAddress, timeout=10)
                if location is None:
                    try:
                        endereco = pycep_correios.consultar_cep(x)
                        inputAddress = endereco['end']+ " , " + endereco['cidade']
                        location = g.geocode(inputAddress, timeout=10)

                        if location is None:
                            lat_log[x] = "NaN"
                        else:
                            lat_log[x] = str(location.latitude)+ "/"+ str(location.longitude)

                    except:
                        lat_log[x] = "NaN"
                
                else:
                    lat_log[x] = str(location.latitude)+ "/"+ str(location.longitude)

    with open('data/lat_log_uni.json', 'w') as file:
        json.dump(lat_log, file)

 

In [ ]:
df_cep['CEP'] = df_cep['CEP'].astype(str)

df_cep = df_cep.replace({"CEP": lat_log})

df_cep.head()

In [ ]:
df_cep.to_csv('data/jupyter_latlog.csv', sep=',')

In [ ]:
#Agrupar os valores importantes para a criação do mapa de calor
#ano = [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
#ano de 2019 foge do escopo, apesar de poder ser incluido na análise

ano = [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018]
semestre = [0,1,2]
time_dict = {}
for x in ano:
    for y in semestre:
        time_dict[str(x)+'_'+str(y)] = []
        df = df_cep.loc[df_cep['Semestre/Ano'] == str(x)+"/"+str(y)]
        lista = df.index.values.astype(int)
        arr = df.drop(['Semestre/Ano', 'CEP'], axis=1).values
        index_names = df.drop(['Semestre/Ano', 'CEP'], axis=1).index
        col_names = df.drop(['Semestre/Ano', 'CEP'], axis=1).columns
        R,C = np.where(arr>0)
        teste = np.zeros_like(index_names[R])
        out_arr = np.column_stack((index_names[R],col_names[C],arr[R,C]))
        df_out = pd.DataFrame(data = out_arr,columns=['row_name','col_name','value'])

        for i in lista:
            for j in out_arr:
                if j[0] == i:
                    time_dict[str(x)+'_'+str(y)].append([df.CEP[i],float(1)])
                    


#### <font color=black> Criação dos mapas de calor</font>

In [ ]:
j = 0
pathto ='/heatmap/'
for i in time_dict.values():

    df = pd.DataFrame(i,columns = ['lat_log','ira'])
    if df.empty:
        j = j +1
    else:
        df[['lat', 'log']] = df.lat_log.str.split('/', expand = True)
        df["ira"] = pd.to_numeric(df["ira"])
        df = df.dropna()
        max_amount = 1
        hmap = folium.Map(location=[-15.7890068,-47.8908946], zoom_start=11, )
        hm_wide = HeatMap( list(zip(df.lat.values, df.log.values, df.ira.values)),
                       min_opacity=0.1,
                       max_val=max_amount,
                       radius=8, blur=7, 
                       max_zoom=1, 
                     )
        hmap.add_child(hm_wide)
        hmap.save(os.path.join(pathto, 'heatmap' +str(j) +'.html'))
        j = j +1

#### <font color=black> Captura dos mapas de calor do formato HTML para o formato PNG</font>

In [ ]:
options = Options()
#options.headless = True

for i in range(33):
    fn='testmap.html'
    tmpurl=pathto+'heatmap'+str(i)+'.html'.format(path=os.getcwd(),mapfile=fn)

    browser = webdriver.Firefox()
    browser.get(tmpurl)
    time.sleep(20)  #Give the map tiles some time to load
    browser.save_screenshot('map'+str(i)+'.png')
    browser.quit()